<a href="https://colab.research.google.com/github/hugomedsa/valuation_acoes_b3/blob/main/Passo_2_Baixar_demonstra%C3%A7%C3%B5es_financeiras_trimestral_(ITR)_e_anual_(DFP)_versao_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


* Configurações Iniciais

In [ ]:
!pip install wget -q
import pandas as pd
from pandas_datareader import data
from time import sleep
import wget, os
import zipfile
import warnings
warnings.filterwarnings('ignore')

  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tip = 'ITR' #caixa alta - ITR ou DFP
a_t = 2023 # ultimo ano com dados
tri = '09' # mes final: 03,06 ou 09

##### Acessando a base de dados e criando arquivos históricos

In [ ]:
 url_base = f'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/{tip}/DADOS/'

* criando uma lista com o nome de todos os arquivos

In [ ]:
arquivos_zip = []
if tip == 'ITR':
  for ano in range(2011,a_t+1):
    arquivos_zip.append(f'{tip.lower()}_cia_aberta_{ano}.zip')
else:
  for ano in range(2010,a_t+1):
    arquivos_zip.append(f'{tip.lower()}_cia_aberta_{ano}.zip')

In [ ]:
arquivos_zip

['itr_cia_aberta_2011.zip',
 'itr_cia_aberta_2012.zip',
 'itr_cia_aberta_2013.zip',
 'itr_cia_aberta_2014.zip',
 'itr_cia_aberta_2015.zip',
 'itr_cia_aberta_2016.zip',
 'itr_cia_aberta_2017.zip',
 'itr_cia_aberta_2018.zip',
 'itr_cia_aberta_2019.zip',
 'itr_cia_aberta_2020.zip',
 'itr_cia_aberta_2021.zip',
 'itr_cia_aberta_2022.zip',
 'itr_cia_aberta_2023.zip']

In [ ]:
# !rm -rf /content/DADOS_CVM

* fazendo o download de todos os arquivos

In [ ]:
for arq in arquivos_zip:
  wget.download(url_base+arq)

* extraindo os arquivos zip

In [ ]:
for archive in arquivos_zip:
  with zipfile.ZipFile(archive, 'r') as zip_file:
    for zip_info in zip_file.infolist():
      for c in ['BPA', 'BPP', 'DFC_MI','DRE']:
        if c in zip_info.filename:
          zip_file.extract(zip_info, f'CVM_{tip}')

* criandos arquivos históricos

In [ ]:
!mkdir DADOS_CVM

In [ ]:
balancos = ['ind','con']

In [ ]:
if f'{tip}_{a_t}' not in os.listdir('/content/drive/MyDrive/'):
  os.mkdir(f'/content/drive/MyDrive/{tip}_{a_t}')

In [ ]:
for bal in balancos:

  nomes = [f'BPA_{bal}', f'BPP_{bal}', f'DRE_{bal}',f'DFC_MI_{bal}']
  dem = []

  for nome in nomes:
    arquivo = pd.DataFrame()
    if tip == 'ITR':
      for ano in range(2011,a_t+1):
        arquivo = pd.concat([arquivo, pd.read_csv(f'CVM_{tip}/{tip.lower()}_cia_aberta_{nome}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
      arquivo.to_csv(f'DADOS_CVM/{tip.lower()}_cia_aberta_{nome}_ate-{a_t}.csv', index=False)
      dem.append(f'DADOS_CVM/{tip.lower()}_cia_aberta_{nome}_ate-{a_t}.csv')
    else:
      for ano in range(2010,a_t+1):
        arquivo = pd.concat([arquivo, pd.read_csv(f'CVM_{tip}/{tip.lower()}_cia_aberta_{nome}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
      arquivo.to_csv(f'DADOS_CVM/{tip.lower()}_cia_aberta_{nome}_ate-{a_t}.csv', index=False)
      dem.append(f'DADOS_CVM/{tip.lower()}_cia_aberta_{nome}_ate-{a_t}.csv')

  dem0 = pd.read_csv(dem[0])[['DT_REFER','DENOM_CIA','CD_CVM','GRUPO_DFP','ORDEM_EXERC','CD_CONTA','DS_CONTA','VL_CONTA']]
  dem1 = pd.read_csv(dem[1])[['DT_REFER','DENOM_CIA','CD_CVM','GRUPO_DFP','ORDEM_EXERC','CD_CONTA','DS_CONTA','VL_CONTA']]
  dem2 = pd.read_csv(dem[2])[['DT_REFER','DENOM_CIA','CD_CVM','GRUPO_DFP','ORDEM_EXERC','CD_CONTA','DS_CONTA','VL_CONTA']]
  dem3 = pd.read_csv(dem[3])[['DT_REFER','DENOM_CIA','CD_CVM','GRUPO_DFP','ORDEM_EXERC','CD_CONTA','DS_CONTA','VL_CONTA']]
  dem_dt = pd.concat([dem0, dem1, dem2, dem3])

  if tip == 'ITR':
    dem_dt['MES'] = dem_dt['DT_REFER'].apply(lambda x: str(x)[5:7])
    dem_juntos = dem_dt[(dem_dt['ORDEM_EXERC'] == 'ÚLTIMO') & (dem_dt['MES'] == tri)]
  else:
    dem_juntos = dem_dt[dem_dt['ORDEM_EXERC'] == 'ÚLTIMO']
  dem_juntos.to_csv(f'/content/drive/MyDrive/{tip}_{a_t}/{tip.lower()}_{bal}_filtrado.csv', index=False)